In [1]:
# Implementation of the simulated annealing algorithm for the 8-tile puzzle.

# A state is a simple list of 9 numbers, a permutation of 0-8. 0 represents the space.

from random import *
from math import *
import numpy as np
import pdb
import time


# Copy a list
def copy_list(list):
    newlist = []
    for x in list:
        newlist.append(x)
    return newlist

# Convert a 0-8 index into a row number
def row(abs_pos):
    return abs_pos//3

# Convert a 0-8 index into a column number
def col(abs_pos):
    return abs_pos%3

# Inverse conversion
def index(row, col):
    return row*3+col

# A nice printout of our state
def print_puzzle(state):
    print (state)
    for r in range(3):
        for c in range(3):
            print (state[index(r,c)],)
        print (" ")

# Returns a list of possible moves from a given state so that we can
# generate a random move. The four possible moves are "N", "S", "E",
# "W" for the four directions, but the function should only return the
# subset of these that are possible in the current configuration.
def all_moves(state):
    
    # get the index of 0
    space = state.index(0) 
    r = row(space)
    c = col(space)
    moves = []
    if r > 0:
        moves.append("N")
        
    if r < 2 :
        moves.append("S")
    
    if c > 0:
        moves.append("W")
        
    if c < 2:
        moves.append("E")
  
    return moves

# Execute a move from the given state. 
def do_move(state, move):
    space = state.index(0)
    r = row(space)
    c = col(space)
    #pdb.set_trace()
    if move == "N":
        move_index = index(r-1, c)
  
    elif move =="S":
        move_index = index(r+1, c)
        
    elif move=="W":
        move_index = index(r, c-1)
        
    elif (move=="E"):
        move_index = index(r, c+1)
        
    state[space],state[move_index] = state[move_index],state[space]
    f.write ("state after move:{}\n".format(state))

# Undo a given move.
def undo_move(state, move):
    f.write("undoing move:{}".format(move))
    print("undoing move:",move)
    if move == "N":
        do_move(state,"S")
        
    elif move =="S":
        do_move(state,"N")
        
    elif move=="W":
         do_move(state,"E")
        
    elif move=="E":
        do_move(state,"W")
        
# will indentify the coordinates of each of goal or initial state values
def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos        
        
# calculate Manhattan distance cost between each digit of puzzle(start state) and the goal state
def manhattan(puzzle, goal):
    
    coordinatePuzzle = coordinates(puzzle)
    coordinateGoal = coordinates(goal)
    
    a = abs(coordinatePuzzle // 3 - coordinateGoal // 3)
    b = abs(coordinatePuzzle % 3 - coordinateGoal % 3)
    mhcost = a + b
    #pdb.set_trace()
    return sum(mhcost)



# will calcuates the number of misplaced tiles in the current state as compared to the goal state
def misplaced_tiles(puzzle,goal):
    #pdb.set_trace()
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0   

#The energy of a state, it is nothing but the objective function
#Manhattan distance or misplaced tiles
def energy(state):
    #pdb.set_trace()
    #manhattan
    if choiceHeuristic == 1:
        return manhattan(state,goal)
    #misplaced    
    elif choiceHeuristic == 2:
         return misplaced_tiles(state,goal)
  
    

    
# return true if it matches the goal else false
def solved(state):
    #print ("checking array equal")
    f.write("goal:{}\n".format(goal))
    f.write("state:{}\n".format(state))
    print("Goal:",goal)  
    print("State:",state)      
    return np.array_equal(state, goal)

# The simulated annealing algorithm.
# We only allow it to run for a given number of maximum moves. The parameter p is a control
# parameter for the rate between the energy and the temperature.
def sim_annealing(initial_state, max_moves, p = 1):
   
    state = initial_state
    temperature = max_moves
    oldE = energy(state)
    #pdb.set_trace()
    while not solved(state) and temperature > 0:
        moves = all_moves(state)
        accepted = False
        while not accepted:
          
            #neighbourhood generation function , we are considering the random choice method
            m = choice(moves)
            f.write("\nChoosing random movement:{}\n".format(m))
            print("\nChoosing random movement:",m) 
            do_move(state, m)
            
            #this is our energy state function
            newE = energy(state)
            
            #pdb.set_trace()
            deltaE = newE - oldE
        
            if deltaE <= 0:
                accepted = True
            else:
                
                # this is our probability generation function
                boltz = exp(-float(p*deltaE)/temperature)
                # A random float between 0 and 1:
                r = random()
                if r <= boltz:
                    accepted = True
            if not accepted:
                undo_move(state,m)
        oldE = newE
        
        # this is our cooling criteria , we are just decreasing it
        temperature = temperature-1
        
        
        
        
def readDataFromFile(): 
    c=1;
    with open("DataInput.txt", "r") as file:
        while (line := file.readline().rstrip()):
            if c==1:
                list_of_integers = list(map(int,line.split(" ")))
                for a in list_of_integers:
                            puzzle.append(a)  
                c=c+1
            elif c==2:
                list_of_integers = list(map(int,line.split(" ")))
                for a in list_of_integers:
                            goal.append(a)  
                c=c+1
            elif c==3:
                heuristic = int(line)
                #pdb.set_trace()
                c=c+1
            
            elif c==4:
                    temp=int(line)
                    
    return temp,heuristic


#choice 1 manhattan, 2 for misplaced tiles
choiceHeuristic = 2
temperature = 0
puzzle = []
goal= []
f = open('output.txt', 'w')


# Main function
def main():
    
    temp, heuristic = readDataFromFile()
    global choiceHeuristic
    choiceHeuristic = heuristic
    
    f.write("Started running simulated annealing:\n")
    print("Started running simulated annealing:\n")      
    startTime = time.time()
    sim_annealing(puzzle, temp, 0.01)
    endTime = time.time()   
    f.write("Completed running simulated annealing:\n")
    print("Completed running simulated annealing:\n")
    
    if solved(puzzle):
        f.write ("Successfully solved:\n")
        print("Successfully solved:\n")  
    else:
        f.write ("Sorry, Could not solved it:\n")
        print ("Sorry, Could not solved it:\n")  
        
    f.write("Execution time:{}".format(endTime-startTime)) 
    print("Execution time:{}".format(endTime-startTime))       
    
main()        


Started running simulated annealing:

Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 0, 5]

Choosing random movement: E
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 5, 0]

Choosing random movement: W
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 0, 5]

Choosing random movement: N
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 0, 4, 7, 6, 5]

Choosing random movement: S
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 0, 5]

Choosing random movement: N
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 0, 4, 7, 6, 5]

Choosing random movement: S
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 0, 5]

Choosing random movement: W
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 0, 7, 5]

Choosing random movement: E
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 7, 0, 5]

Choosing random movement: W
Goal: [1, 2, 3, 8, 0, 4, 7, 6, 5]
State: [2, 8, 3, 1, 6, 4, 0, 7, 5]

Choosing r